In [1]:

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
import scipy.sparse as sp
import numpy as np
from ngsolve.fem import LeviCivitaSymbol, Einsum

from customOperators import GGrad, GCurl, GDiv

In [2]:
# Relevant parameters to play around with

order = 1 # mesh order
max_h = 0.2 # mesh size
C_w = 1000# penalty term weight
d = 3 # mesh dimension..

In [3]:
# 1. Define the Geometry
geo = CSGeometry()
box = OrthoBrick(Pnt(-1, -1, -1), Pnt(1, 1, 1))
geo.Add(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))
# Draw(mesh)

In [4]:
V_k = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
V_km1 = H1(mesh, order=order)

prodSpc = V_k * V_km1

omega, sigma = prodSpc.TrialFunction()
eta, tau = prodSpc.TestFunction()

In [5]:
a = BilinearForm(prodSpc)

a += sigma * tau * dx
a += - omega * grad(tau) * dx

a +=  grad(sigma) * eta * dx
a +=  curl(omega) * curl(eta) * dx

Lets unpack the first boundary term from (2c):

$$
+ \, \int_{\Gamma} \text{tr}(\star d\omega_h) \wedge \text{tr}(\eta_h) \tag{2c.1}
$$

If we consider the k-forms in 3D to be: 

$$
\omega , \eta \in \Lambda^1(\Omega) \, \Longleftrightarrow \, \omega, \eta \in H(\text{curl}, \Omega)
$$

We can plug in the according vector calculus operations and transform them with help from some identities:

$$
\begin{aligned}
&= \int_{\Gamma} (\text{n} \times \nabla \times \omega_h) \cdot \eta_h \\[10pt]

&= \int_{\Gamma} (\nabla \times \omega_h) \cdot (\eta_h \times \text{n})
\end{aligned}
$$



In [6]:
n = specialcf.normal(mesh.dim)

integrand_2c1 = CF(curl(omega) * Cross(eta, n))
a += integrand_2c1 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))


And the second boundary term from (2c):

$$
\begin{aligned}
&+ \int_{\Gamma} \text{tr} ( \omega_h ) \wedge \text{tr} ( \star d\eta_h ) \tag{2c.2} \\[10pt]

&=\int_{\Gamma} (\omega_h \times \text{n}) \cdot (\nabla \times \eta_h)
\end{aligned}
$$

In [7]:
integrand_2c2 = CF( Cross(omega, n) * curl(eta) )
a += integrand_2c2 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

And the pentalty or stabilization term:

$$
+ \, \frac{C_w}{h} \langle \text{tr}\, \omega, \text{tr}\, \eta \rangle_{L^2(\Gamma)} \tag{2d.1}
$$

In [8]:
h = specialcf.mesh_size
a += (C_w / h) * Cross(omega, n) * Cross(eta, n) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

In [9]:
A = 0.05  # Amplitude of the pulse
sigma_pulse = 0.2  # Width of the Gaussian pulse
r0 = (0.5, 0.5, 0.5)  # Center of the Gaussian pulse
n_pulse = (1, 0, 0)  # Direction of the pulse (unit vector)

gauss_pulse = CF((
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[0],
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[1],
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[2]
))

omega_m = CF((sin(pi*x)*sin(pi*y)*sin(pi*z), 
              sin(pi*x)*sin(pi*y)*sin(pi*z), 
              sin(pi*x)*sin(pi*y)*sin(pi*z)))

omega_m = gauss_pulse
hL_omega = GCurl(GCurl(omega_m, d), d) - GGrad(GDiv(omega_m, d), d)


f = LinearForm(prodSpc)
f += hL_omega * eta * dx

In [10]:
a.Assemble()
f.Assemble()

rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
#cond_nr = np.linalg.cond(A.todense())

sol = GridFunction(prodSpc)
res = f.vec-a.mat * sol.vec
inv = a.mat.Inverse(freedofs=prodSpc.FreeDofs(), inverse="pardiso")
sol.vec.data += inv * res

gf_omega , gf_sigma = sol.components

curl_omega = curl(gf_omega)
grad_sigma = grad(gf_sigma)

print("Matrix dimensions:", a.mat.height, "x", a.mat.width)
#print("Matrix Condition Number: ", cond_nr)
print("Residual: ", Norm(res))
print("L2 Error omega: ", sqrt(Integrate((gf_omega - omega_m)**2, mesh)))
print("L2 Error curl(omega): ", sqrt(Integrate((curl(gf_omega) - GCurl(omega_m, d))**2, mesh)))
print("L2 Error sigma:: ", sqrt(Integrate((gf_sigma + GDiv(omega_m, d))**2, mesh)))
print("L2 Error grad(sigma): ", sqrt(Integrate((grad(gf_sigma) + GGrad(GDiv(omega_m, d), d))**2, mesh)))



Matrix dimensions: 8929 x 8929
Residual:  4.467980713022058e-09
L2 Error omega:  0.003317477947890036
L2 Error curl(omega):  0.01939065792961446
L2 Error sigma::  0.009372684261690498
L2 Error grad(sigma):  0.15614502330415353


In [11]:
Draw(gf_omega)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene